In [52]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import time

In [53]:
driver = webdriver.Chrome()
driver.get("https://www.jumia.com.eg/catalog/?q=phone")

In [54]:
height=0
for i in range (11):
    height=height+500
    driver.execute_script(f"window.scrollTo(0,{height});")
    time.sleep(1)

In [55]:
products = driver.find_elements(By.XPATH, "//article[@class='prd _fb col c-prd']")
product_data = []

In [56]:
for product in products:
    name = product.find_element(By.XPATH, ".//h3[@class='name']").text
    price = product.find_element(By.XPATH, ".//div[@class='prc']").text.replace("EGP ", "")

    
    image_element = product.find_element(By.XPATH, ".//img[@class='img']")
    if image_element and 'src' in image_element.get_attribute('outerHTML'):
        image_link = image_element.get_attribute('src')

       
        if image_link.startswith('data:image'):
            print(f"Skipping image with data URL: {image_link}")
            continue

        response = requests.get(image_link, stream=True)
        with open(f'img-{len(product_data)+1}.jpg', 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
    else:
        image_link = "No image"

    
    try:
        phone_ratings = product.find_element(By.XPATH, ".//div[@class='stars _s']").text
        phone_rating_texts = [rating.split()[0] for rating in phone_ratings.split() if rating[0].isdigit()]
        rating = phone_rating_texts[0] if phone_rating_texts else "No rating"
    
    except:
        rating = "No rating"

    if rating != "No rating":
        product_data.append({
            "Product Name": name,
            "Product Price (EGP)": price,
            "Product Rating": rating,
            "Product Image": image_link
            
        })



In [57]:
df = pd.DataFrame(product_data)

In [58]:
df.to_csv('jumia_products_complete.csv', index=False, encoding='utf-8')